In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pandas as pd
from dev_interaction_util import *
from DevCvAnalysis import DevCvAnalysis

In [8]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


The basic structure is the following:

1. Run the following cross-validated analyses:
   1. Predicting change by condition only
   2. Predicting change by condition and neural and behavioral measures
   3. Predicting change by condition, neural and behavioral measures, and their interactions
2. Measure the predictivity of the three models above using anova
3. Repeat the steps above separately for three outcome variables, change in: FFQ, ASA-24, and BFP
4. 

Tasks to do to get this job done (not in order):

1. Write the analysis pipeline above
2. Get the neural data
3. Get the behavioral data


We have the behavioral data. Do we have the neural data already?

What could we delegate here? Behavioral data we already have. We have mostly writen the analysis pipeline. The neural data could be passed on.

# Get the data

## Set up data


In [9]:
%load_ext autoreload
%autoreload 2

from InterventionCVManager import *

dropbox_data_dir = config['dropbox_data_dir']

icvm = InterventionCVManager(dropbox_data_dir)
#icvm.mode = 'full_pipeline_test'
#icvm.mode = 'fast_pipeline_test'
icvm.mode=  'fast_analysis'
#icvm.mode = 'full_analysis'
icvm.group_mode = 'exclude_mckenzie'
#dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%load_ext autoreload
%autoreload 2
dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV022', 'DEV007', 'DEV032', 'DEV002'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


run_real_analysis is True, so we're not randomizing the outcomes. 
(157, 33)


# Predict change

### Set up sets of variables to run

First, we set up a function that loops runs the scoring loop above (which does one cross-validation analysis), and the nadditionally:
- selects the best model based on the overall results
- Runs a final fit
- presents model results

Now, we include functions that compare models with and without individual differences and interactions.

Should manually verify that in the following list, the intervention_group allocations are randomized (if we're running a test run) or that they are accurate (if it's not a test run).

In [13]:
pd.concat([
    dev_cv_analysis.outcome_measures['SID'],
      dev_cv_analysis.group_assignments
],axis=1)

,SID,intervention_group
0,DEV004,umpqua
1,DEV005,umpqua
2,DEV008,umpqua
3,DEV009,umpqua
4,DEV011,umpqua
...,...,...
152,DEV305,willamette
153,DEV306,willamette
154,DEV308,willamette
155,DEV309,umpqua


In [14]:
dev_cv_analysis.outcome_measures.columns

Index(['SID', 'bf', 'NUTRIENT_RICH_FOODS_INDEX_2wkAverage',
       'ANTINUTRIENT_DENSITY_2wkAverage', 'total_calorie'],
      dtype='object')

In [15]:
condition_cols = dev_cv_analysis.group_assignment_onehots.columns.tolist()
inddiff_cols = dev_cv_analysis.get_predictors_main_names()

interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_only': condition_cols,
    'condition_inddiff': condition_cols + inddiff_cols,
    'condition_inddiff_interactions': condition_cols + inddiff_cols + interaction_cols
}


outcome_vars_to_try = [ 'NUTRIENT_RICH_FOODS_INDEX_2wkAverage']

In [16]:
print(", ".join(dev_cv_analysis.get_predictor_data().columns))

EDM, BIS_11, PCS, ACES_sum, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, NCS_total, TESQ_E_sum, SRHI_healthy_minus_unhealthy, RTFS_f1_minus_f2, cancer_promoting_minus_preventing_FCI, age365, education_own, household_income_per_person, SST_PostErrorSlowW1_mean, SST_mean_ssrt_0, ROC_Crave_Regulate_Minus_Look, ROC_Crave_Minus_Neutral, WTP_unhealthy_minus_healthy, wtp_liked_value_association-test_z_FDR_0.01, roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01, roc_reappraiseCrave_multivariate_regulation, sst_CorrectGo_striatum_joint_mask, sst_FailedStop_motor_control_striatum_joint_mask, sst_CorrectGoFollowingFailedStop_striatum_joint_mask, Planning_aggregate, Restraint_aggregate, IMI_effort_importance_aggregate, wtp_roc_koban_kober_craving_combined, birthsex_factor_Male, umpqua, EDM*umpqua, BIS_11*umpqua, PCS*umpqua, ACES_sum*umpqua, BFI_agreeableness*umpqua, BFI_conscientiousness*umpqua, BFI_extraversion*umpqua, BFI_neuroticism*umpqua

In [17]:
print(", ".join(dev_cv_analysis.outcome_measures.columns))

SID, bf, NUTRIENT_RICH_FOODS_INDEX_2wkAverage, ANTINUTRIENT_DENSITY_2wkAverage, total_calorie


## Nutrient density

In [18]:
model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_RICH_FOODS_INDEX_2wkAverage', dev_cv_analysis)

## condition_only

loading raw data


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:998: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV280', 'DEV022', 'DEV007', 'DEV032', 'DEV002'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')
run_real_analysis is True, so we're not randomizing the outcomes. 
(157, 33)
 attempting to predict NUTRIENT_RICH_FOODS_INDEX_2wkAverage with 1 predictors in the set condition_only
predictors in that set are umpqua
outer split0
Only one feature, so skipping feature selection
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 fold

doing permutation test on importance; this may take time.


...

Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Ridge())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Ridge())])
{'feature_selection__score_func': [<function f_regression at 0x17d3f99e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Time elapsed for Ridge: 0.78 seconds
Pipeline(steps=[('scaler', StandardScaler()), ('estimator', Lasso())])
{'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 2 candidates, totalling 8 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selection', SelectKBest()), ('estimator', Lasso())])
{'feature_selection__score_func': [<function f_regression at 0x17d3f99e0>], 'feature_selection__k': [5, 10, 15], 'estimator__alpha': [0.2, 0.99]}
Fitting 4 folds for each of 6 can

/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/benjaminsmith/opt/anaconda3/envs/dataanalysis/lib/python3.11/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


doing permutation test on importance; this may take time.


,predictor,coef,feature_importance,fa_abs
12,SRHI_healthy_minus_unhealthy,None,0.005376,0.005376
21,ROC_Crave_Minus_Neutral,None,0.004932,0.004932
30,Restraint_aggregate,None,0.004903,0.004903
14,cancer_promoting_minus_preventing_FCI,None,0.004864,0.004864
37,ACES_sum*umpqua,None,0.004326,0.004326
54,ROC_Crave_Minus_Neutral*umpqua,None,0.004068,0.004068
48,age365*umpqua,None,0.004061,0.004061
45,SRHI_healthy_minus_unhealthy*umpqua,None,0.003918,0.003918
43,NCS_total*umpqua,None,0.003788,0.003788
39,BFI_conscientiousness*umpqua,None,0.003718,0.003718


{'condition_only': -0.05667967632562996, 'condition_inddiff': -0.10543937856195687, 'condition_inddiff_interactions': -0.0478980411159162}
